<a href="https://colab.research.google.com/github/ALTH-Alma/Deep-model-test/blob/main/deep_sanne_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Arquitectura Deep-SANNE**

**Input Layer:** La capa de entrada recibe secuencias con un total de 24 pasos temporales, cada uno con dimensiones de 30x30.

**Primera Capa Densamente Conectada:** Tras la capa de entrada, hay una capa densamente conectada (fully-connected) con activación lineal, que aumenta el número de canales a 64, manteniendo los 24 pasos temporales y las dimensiones espaciales de 30x30.

**Convolutional LSTM Layer:** Esta capa consiste en 24 celdas LSTM que realizan operaciones convolucionales, aumentando el número de canales a 128. El kernel convolucional utilizado es de 2D con un tamaño de 4x4, moviéndose a través de las dimensiones espaciales (n, m). La función de activación utilizada en esta capa es la unidad lineal rectificada (ReLU).

**Segunda Capa Densamente Conectada:** Solo la salida de la última celda LSTM se considera y se alimenta a otra capa densamente conectada con activación lineal y las mismas dimensiones aumentadas de 30x30x128.

**Capa Densamente Conectada Final:** Finalmente, hay una última capa densamente conectada que reduce las dimensiones de nuevo a 30x30, sirviendo como la salida de la red.

In [ ]:
#Prueba
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#get dataset de prueba ---- Probar cuanto aguanta de memoria
'''(train_images, train_vectorial), (test_images, test_vectorial) = some..
 plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

plt.figure()
plt.imshow(train_vectorial[0])
plt.colorbar()
plt.grid(False)
plt.show() '''

#  probar cuanto cae en memoria de un dispositivo
#Construir modelo: multidispositivos

model = tf.keras.Sequential([
    #Imput layer t*(n_1*m_1)
    tf.keras.Input(
        shape=(24, 30, 30, 1) #Considerando entrada de 24 pasos de tiempo, todas de 30 x 30, canales color (3)
    ),
    #First Densely connected layer t*(n_2*m_2*l_2) / l_2=64  (configuracion de un imput por canal - averiguar)
    tf.keras.layers.Dense(
        units = 64, # aumenta el numero de canales a 64
        activation='linear' # función de activacion lineal
    ),
    #2D Convolutional LSTM layer
    tf.keras.layers.ConvLSTM2D(
        filters=128,  # Canaless de salida 128
        kernel_size=(4, 4),  # tamaño del kernel 4x4
        strides=(1, 1),  # por defecto
        padding='same',  # mantener dimensionalidad entrada salida
        activation='relu',  # función de activacion ReLU
        return_sequences=False,  # retorna secuencias para permitir conexiones secuenciales  (devuelve solo la ultima salida en la secuencia de salidas)
        recurrent_activation='hard_sigmoid',  # 'hard_sigmoid' es una opción común para recurrent_activation
    ),
    #Second Densely connected layer (n_4*m_4*l_4) / l_4=128  Only the output of the last LSTM cell is considered return_sequences=False cumple
    tf.keras.layers.Dense(
        units = 128, # aumenta el numero de canales a 128
        activation='linear' # función de activacion lineal
    ),
    #Finally Densely Connected Layer (Output Layer)
    tf.keras.layers.Dense(units=1, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001))
])

**Primera capa densamente conectada:**

Aumenta el número de canales a 64.

Mantiene los 24 pasos temporales y las dimensiones espaciales de 30x30.

Utiliza una función de activación lineal.

**Capa de entrada:**

Recibe secuencias de 24 pasos temporales.

Cada paso tiene dimensiones espaciales de 30x30.

Posee 3 canales para el color.

**Capa ConvLSTM2D:**

Contiene 24 celdas LSTM que realizan operaciones convolucionales.

Aumenta el número de canales a 128.

Utiliza un kernel convolucional 2D de tamaño 4x4.

Se mueve a través de las dimensiones espaciales (n, m).

Posee una función de activación ReLU.

Retorno de secuencias: Falso, solo se considera la última salida de la secuencia.

Activación recurrente: 'hard_sigmoid', una opción común para redes LSTM.

**Segunda capa densamente conectada:**

Solo se utiliza la salida de la última celda LSTM.

Aumenta las dimensiones a 30x30x128.

Posee una función de activación lineal.

**Capa final:**

Reduce las dimensiones a 30x30.

Genera la salida final de la red.

**Función de activación tanh (preguntar):
Rango de salida: [-1, 1].
Preserva la magnitud y dirección de los campos vectoriales.
Adecuada para la representación de campos vectoriales con valores positivos y negativos.

In [ ]:
import torch
import torch.nn as nn
import helper
import sys
import time
import re
import numpy as np
import matplotlib as plt

In [ ]:
f1=nn.Flatten()
l1=nn.Linear(28*28, 300)
r1=nn.ReLU()
l2=nn.Linear(300, 100)
r2=nn.ReLU()
l3=nn.Linear(100, 30)
s3=nn.Softmax()
model = nn.Sequential(\
    f1,
    l1,
    r1,
    l2,
    r2,
    l3,
    s3, \
)

print("Model: ", model)

Model:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=300, bias=True)
  (2): ReLU()
  (3): Linear(in_features=300, out_features=100, bias=True)
  (4): ReLU()
  (5): Linear(in_features=100, out_features=30, bias=True)
  (6): Softmax(dim=None)
)


In [2]:
import torch
from torch import nn
from torch.nn import functional as F



In [3]:
#First Densely Layer
class DenseLayer(nn.Module):
    def __init__(self, units, activation):
        super(DenseLayer, self).__init__()
        self.units = units
        self.activation = activation

        self.linear = nn.Linear(1, units)

    def forward(self, x):
      # dimensiones de entrada --averiguar
        #x = x.view(-1, 1) # considerando una dimension
        x = self.linear(x)
        x = self.activation(x)
        return x

dense_layer_1 = DenseLayer(units=64, activation=F.linear)


In [5]:
# Convolutional LTSM Layer

# Definir celda LSTM convolucional
class ConvLSTMCell(nn.Module):
    def __init__(self, filters, kernel_size):
        super(ConvLSTMCell, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size

        # Puertas de entrada y olvido
        self.conv_gate = nn.Conv2d(filters * 4, filters * 4, kernel_size=kernel_size, padding=1)
        # Puerta de salida
        self.conv_out = nn.Conv2d(filters * 2, filters, kernel_size=kernel_size, padding=1)

    def forward(self, x, h_cell, c_cell):
        # Concatenar datos de entrada con estado oculto anterior h_cell
        concat = torch.cat([x, h_cell], dim=1)

        # Aplicar la capa de convolucion
        gates = self.conv_gate(concat)

        # Divider las dalidas de la convolucion en las puertas de entrada, olvido, salida y puerta de cekda
        i, f, o, g = torch.split(gates, self.filters, dim=1)
        # Activacion sig
        i = F.sigmoid(i)
        f = F.sigmoid(f)
        o = F.sigmoid(o)
        # Activacion tanh para puerta de celda
        g = F.tanh(g)

        # Actualizar estado de la celda
        c_new = f * c_cell + i * g

        # Actualizar salida. Funcion de activacion tanh a la concatenacion de datos de entrada y nuevos estados... cambiar dim
        out = o * F.tanh(self.conv_out(torch.cat([x, c_new], dim=1)))

        # Returnar salida y estado actual
        return out, c_new


# Definir capa LSTM convolucional en 2D
class ConvLSTM2D(nn.Module):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='same', activation='relu'):
        super(ConvLSTM2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.activation = activation

        self.conv_lstm_cell = ConvLSTMCell(filters, kernel_size) # crear instancia de celda LSTM para realizar la operacion LSTM convolucional

    def forward(self, x, h_state=None):
        batch_size, seq_len, in_channels, height, width = x.size()

        # Inicializar estado oculto y celda en caso de no entregarse
        if h_state is None:
            h_state = (torch.zeros(batch_size, self.filters, height, width, device=x.device),
                       torch.zeros(batch_size, self.filters, height, width, device=x.device))

        hidden_states = []
        cell_states = []

        # Iterar sobre los elementos de la secuencia de entreda
        for t in range(seq_len):
            x_t = x[:, t, :, :]
            h_t, c_t = self.conv_lstm_cell(x_t, *h_state) #Aplicaar celda convolucionar a cada elemento
            hidden_states.append(h_t)
            cell_states.append(c_t)
            h_state = (h_t, c_t)  # Actualizar estado

        hidden_states = torch.stack(hidden_states, dim=1)  # Almacenar estados ocultos
        cell_states = torch.stack(cell_states, dim=1)  # Almacenar estados de celda

        # Solo retorna el ultimo resultado (return_sequences=False)
        return hidden_states[:, -1, :, :], cell_states[:, -1, :, :]

conv_lstm2d_layer = ConvLSTM2D(
    filters=128,
    kernel_size=(4, 4),
    strides=(1, 1),
    padding='same',
    activation=F.relu,
)


In [6]:
#Second Densely Layer
class DenseLayer2(nn.Module):
    def __init__(self, units, activation):
        super(DenseLayer2, self).__init__()
        self.units = units
        self.activation = activation

        self.linear = nn.Linear(128, units)

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

dense_layer_2 = DenseLayer2(units=128, activation=F.linear)


In [7]:
#Output Layer
class OutputLayer(nn.Module):
    def __init__(self):
        super(OutputLayer, self).__init__()
        self.linear = nn.Linear(128, 1)

    def forward(self, x):
        x = self.linear(x)
        x = torch.tanh(x)
        return x

output_layer = OutputLayer()


In [8]:
#Model Create
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.dense_layer_1 = dense_layer_1
        self.conv_lstm2d_layer = conv_lstm2d_layer
        self.dense_layer_2 = dense_layer_2
        self.output_layer = output_layer

    def forward(self, x):
        x = self.dense_layer_1(x)
        x = self.conv_lstm2d_layer(x)
        x = self.dense_layer_2(x)
        x = self.output_layer(x)
        return x

model = Model()


In [9]:
print(model)

Model(
  (dense_layer_1): DenseLayer(
    (linear): Linear(in_features=1, out_features=64, bias=True)
  )
  (conv_lstm2d_layer): ConvLSTM2D(
    (conv_lstm_cell): ConvLSTMCell(
      (conv_gate): Conv2d(512, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (conv_out): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    )
  )
  (dense_layer_2): DenseLayer2(
    (linear): Linear(in_features=128, out_features=128, bias=True)
  )
  (output_layer): OutputLayer(
    (linear): Linear(in_features=128, out_features=1, bias=True)
  )
)
